<a href="https://colab.research.google.com/github/tonytoon/ivylearnnotebooks/blob/main/outcomes_enrollment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script will generate an SIS feed file for outcomes organizations. This feed file will generate all organizations, sections, and enrollments.

It is run as a diffed update which will mark any enrollments that do not exist in a future run against the same diff data ID as inactive.

To configure for each term, ensure that outcomes_courses, term, and term_suffix are all properly entered.

It will generate a .zip file containing three .csv files which can be uploaded to the sis feed section of the admin panel in Canvas.

In [ ]:
!pip install canvasapi

In [ ]:
from canvasapi import Canvas # used for interfacing with canvas
from zipfile import ZipFile
import csv
# Canvas API URL
# ALWAYS test scripts on the beta server until you are 100% that it will work properly.
#API_URL = "https://.instructure.com" # prod
API_URL = "https://.beta.instructure.com" # beta

# Canvas API key
#API_KEY = "" #prod
API_KEY = "" #beta

# Initialize a new Canvas object
canvas = Canvas(API_URL, API_KEY)
account = canvas.get_account(417) # 
outcomes_account = canvas.get_account(16179) 

In [ ]:
# list of courses needing outcomes organizations
outcomes_courses = ["ACCT101", "BIOL101", "BUSI279", "BUSN101", "CPIN279", "CRIM260", "HIST101", "HLHS101", "HLHS105", "HUMS279", "MATH123", "PSYC101", "SOCI111"]

# term/acct information
term = 3265
term_suffix = "202210"

########### nothing below this line should need to modified per term

# account / org info
outcomes_acct = "SW-OUTCOMES-ORGZ" 
org_suffix = "-OUTCOMES-SW-ORGZ-" + term_suffix + "-INI"

# part of term codes
termcodes = ["81X", "82X", "16R", "ZDC"]

# diffing options
# see: https://canvas.instructure.com/doc/api/file.sis_csv.html
diff_code = "OUTCOMES-" + term_suffix
diff_remaster = False
clear_sticky = True
override_sticky = True

In [ ]:
# file handling
courses_filename = "courses.csv"
courses_header = ["course_id","short_name","long_name","account_id","status"]

sections_filename = "sections.csv"
sections_header = ["section_id", "course_id", "name", "status"]

enrollments_filename = "enrollments.csv"
enrollments_header = ["course_id","user_id","role","section_id","status","limit_section_privileges"]


In [ ]:
# generate courses
courses = []

# we want source courses of: ACCT101 in P1 sub-account
# to create a single org of: ACCT101-OUTCOMES-SW-ORGZ-202210-INI

for course_code in outcomes_courses:
  courses.append([f"{course_code}{org_suffix}",f"{course_code}{org_suffix}",f"{course_code}{org_suffix}",outcomes_acct,"ACTIVE"])

In [ ]:
# write courses.csv
with open(courses_filename, 'w', encoding='UTF8', newline='') as csvfile:
  csvwriter = csv.writer(csvfile)
  csvwriter.writerow(courses_header)
  csvwriter.writerows(courses)

In [ ]:
# generate sections
sections = []

# we want a source course of:       ACCT101-0AC-C1-202210-VI-81X
# to create an outcomes section of: ACCT101-0AC-C1-202210-VI-OUTCOMES-81X
# in the organization:              ACCT101-OUTCOMES-SW-ORGZ-202210-INI

for course_code in outcomes_courses:
  src_courses = account.get_courses(search_term=course_code, enrollment_term_id=term)
  for c in src_courses:
    section = c.course_code
    for tc in termcodes:
      section = section.replace(tc, f"OUTCOMES-{tc}")
    sections.append([section, f"{course_code}{org_suffix}",section,"ACTIVE"])


In [ ]:
# write sections.csv
with open(sections_filename, 'w', encoding='UTF8', newline='') as csvfile:
  csvwriter = csv.writer(csvfile)
  csvwriter.writerow(sections_header)
  csvwriter.writerows(sections)

In [ ]:
# generate enrollments
enrollments = []

for course_code in outcomes_courses:
  src_courses = account.get_courses(search_term=course_code, enrollment_term_id=term)
  target_org = f"{course_code}{org_suffix}"
  for c in src_courses:
    section = c.course_code
    for tc in termcodes:
      section = section.replace(tc, f"OUTCOMES-{tc}")

    # add students
    s_users = c.get_users(enrollment_type=['student'])
    for s in s_users:
      enrollments.append([target_org,s.sis_user_id,"student",section,"active","TRUE"])

    # add instructors
    # we use the Outcomes Instructor role in the outcomes org
    # for custom permissions
    t_users = c.get_users(enrollment_type=['teacher'])
    for t in t_users:
      enrollments.append([target_org,t.sis_user_id,"Outcomes Instructor",section,"active","TRUE"])

In [ ]:
# write enrollments.csv
with open(enrollments_filename, 'w', encoding='UTF8', newline='') as csvfile:
  csvwriter = csv.writer(csvfile)
  csvwriter.writerow(enrollments_header)
  csvwriter.writerows(enrollments)

In [ ]:
# create zip file to upload
zipObj = ZipFile('outcomes_enrollments.zip', 'w')
zipObj.write(courses_filename)
zipObj.write(sections_filename)
zipObj.write(enrollments_filename)
zipObj.close()

In [ ]:
# upload sis feed
# check https://canvas.instructure.com/doc/api/file.sis_csv.html
# since we use diffing, we have to use the api
account.create_sis_import("outcomes_enrollments.zip", diffing_data_set_identifier=diff_code, diffing_drop_status="inactive", diffing_remaster_data_set=diff_remaster, override_sis_stickiness=override_sticky, clear_sis_stickiness=clear_sticky)